# Build movie recommendation system

## Import libraries

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import ast 
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD, evaluate

import warnings; warnings.simplefilter('ignore')

## 2. Load dataset

In [63]:
credits = pd.read_csv('credits.csv')
keywords = pd.read_csv('keywords.csv')
links_small = pd.read_csv('links_small.csv')
md = pd.read_csv('movies_metadata.csv')
ratings = pd.read_csv('ratings_small.csv')

C:\Users\91720\AppData\Local\Temp\ipykernel_11760\2252783192.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  md = pd.read_csv('movies_metadata.csv')


##  Understand dataset

#### Credits dataframe

In [18]:
credits.head()

cast  \
0  [{'cast_id': 14, 'character': 'Woody (voice)',...   
1  [{'cast_id': 1, 'character': 'Alan Parrish', '...   
2  [{'cast_id': 2, 'character': 'Max Goldman', 'c...   
3  [{'cast_id': 1, 'character': "Savannah 'Vannah...   
4  [{'cast_id': 1, 'character': 'George Banks', '...   

                                                crew     id  
0  [{'credit_id': '52fe4284c3a36847f8024f49', 'de...    862  
1  [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...   8844  
2  [{'credit_id': '52fe466a9251416c75077a89', 'de...  15602  
3  [{'credit_id': '52fe44779251416c91011acb', 'de...  31357  
4  [{'credit_id': '52fe44959251416c75039ed7', 'de...  11862

In [19]:
credits.columns

Index(['cast', 'crew', 'id'], dtype='object')

* **cast:** Information about casting. Name of actor, gender and it's character name in movie
* **crew:** Information about crew members. Like who directed the movie, editor of the movie and so on. 
* **id:** It's movie ID given by TMDb

In [20]:
credits.shape

(45476, 3)

In [21]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


#### Keywords dataframe

In [22]:
keywords.head()

id                                           keywords
0    862  [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...
1   8844  [{'id': 10090, 'name': 'board game'}, {'id': 1...
2  15602  [{'id': 1495, 'name': 'fishing'}, {'id': 12392...
3  31357  [{'id': 818, 'name': 'based on novel'}, {'id':...
4  11862  [{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...

In [23]:
keywords.columns

Index(['id', 'keywords'], dtype='object')

* **id:** It's movie ID given by TMDb
* **Keywords:** Tags/keywords for the movie. It list of tags/keywords 

In [24]:
keywords.shape

(46419, 2)

In [25]:
keywords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46419 entries, 0 to 46418
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        46419 non-null  int64 
 1   keywords  46419 non-null  object
dtypes: int64(1), object(1)
memory usage: 725.4+ KB


#### Link dataframe

In [26]:
links_small.head()

movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0

In [27]:
links_small.columns

Index(['movieId', 'imdbId', 'tmdbId'], dtype='object')

* **movieId:** It's serial number for movie
* **imdbId:** Movie id given on IMDb platform
* **tmdbId**: Movie id given on TMDb platform

In [28]:
links_small.shape

(9125, 3)

In [29]:
links_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB


#### Metadata dataframe

In [30]:
md.iloc[0:3].transpose()

0  \
adult                                                              False   
belongs_to_collection  {'id': 10194, 'name': 'Toy Story Collection', ...   
budget                                                          30000000   
genres                 [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
homepage                            http://toystory.disney.com/toy-story   
id                                                                   862   
imdb_id                                                        tt0114709   
original_language                                                     en   
original_title                                                 Toy Story   
overview               Led by Woody, Andy's toys live happily in his ...   
popularity                                                     21.946943   
poster_path                             /rhIRbceoE9lR4veEXuwCC2wARtG.jpg   
production_companies      [{'name': 'Pixar Animation Studios', 'id': 3}]   
production_countries   [{'iso_3166_1': 'US', 'name': 'United States o...   
release_date                                                  1995-10-30   
revenue                                                      373554033.0   
runtime                                                             81.0   
spoken_languages                [{'iso_639_1': 'en', 'name': 'English'}]   
status                                                          Released   
tagline                                                              NaN   
title                                                          Toy Story   
video                                                              False   
vote_average                                                         7.7   
vote_count                                                        5415.0   

                                                                       1  \
adult                                                              False   
belongs_to_collection                                                NaN   
budget                                                          65000000   
genres                 [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
homepage                                                             NaN   
id                                                                  8844   
imdb_id                                                        tt0113497   
original_language                                                     en   
original_title                                                   Jumanji   
overview               When siblings Judy and Peter discover an encha...   
popularity                                                     17.015539   
poster_path                             /vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg   
production_companies   [{'name': 'TriStar Pictures', 'id': 559}, {'na...   
production_countries   [{'iso_3166_1': 'US', 'name': 'United States o...   
release_date                                                  1995-12-15   
revenue                                                      262797249.0   
runtime                                                            104.0   
spoken_languages       [{'iso_639_1': 'en', 'name': 'English'}, {'iso...   
status                                                          Released   
tagline                        Roll the dice and unleash the excitement!   
title                                                            Jumanji   
video                                                              False   
vote_average                                                         6.9   
vote_count                                                        2413.0   

                                                                       2  
adult                                                              False  
belongs_to_collection  {'id': 119050, 'name': 'Grumpy Old Men Collect...  
budget                                      

In [31]:
md.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [32]:
md.shape

(45466, 24)

In [33]:
md.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

#### Ratings dataframe

In [34]:
ratings.head()

userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179
2       1     1061     3.0  1260759182
3       1     1129     2.0  1260759185
4       1     1172     4.0  1260759205

In [35]:
ratings.columns

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')

* **userId:** It is id for User
* **movieId:** It is TMDb movie id.
* **rating:** Rating given for the particular movie by specific user
* **timestamp:** Time stamp when rating has been given by user

In [36]:
ratings.shape

(100004, 4)

In [37]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100004 entries, 0 to 100003
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100004 non-null  int64  
 1   movieId    100004 non-null  int64  
 2   rating     100004 non-null  float64
 3   timestamp  100004 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [38]:
md['genres'] = md['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i[
    'name'] for i in x] if isinstance(x, list) else [])

In [39]:
# this is V
vote_counts = md[md['vote_count'].notnull()]['vote_count'].astype('int')

# this is R
vote_averages = md[md['vote_average'].notnull()]['vote_average'].astype('int')

# this is C
C = vote_averages.mean()
C

5.244896612406511

* The next step, we need to determine an appropriate value for `m`, the minimum votes required to be listed in the chart. 

* We will use 95th percentile as our cutoff. In other words, for a movie to feature in the charts, it must have more votes than at least 95% of the movies in the list.



In [40]:
m = vote_counts.quantile(0.95)
m

434.0

In [41]:
# Pre-processing step for getting year from date by splliting it using '-'

md['year'] = pd.to_datetime(md['release_date'], errors='coerce').apply(
    lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [42]:
qualified = md[(md['vote_count'] >= m) & 
               (md['vote_count'].notnull()) & 
               (md['vote_average'].notnull())][['title', 
                                                'year', 
                                                'vote_count', 
                                                'vote_average', 
                                                'popularity', 
                                                'genres']]

qualified['vote_count'] = qualified['vote_count'].astype('int')
qualified['vote_average'] = qualified['vote_average'].astype('int')
qualified.shape

(2274, 6)

* Therefore, to qualify to be considered for the chart, a movie has to have at least __434 votes__ on TMDB. 
* We also see that the __average rating__ for __a movie on TMDB__ is __5.244 on a scale of 10__. 
* Here, only __2274 movies__ are qualify to be on our chart.

In [43]:
def weighted_rating(x):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [44]:
qualified['wr'] = qualified.apply(weighted_rating, axis=1)

In [45]:
qualified = qualified.sort_values('wr', ascending=False).head(250)

**Top Movies**

In [46]:
qualified.head(15)

title  year  vote_count  \
15480                                          Inception  2010       14075   
12481                                    The Dark Knight  2008       12269   
22879                                       Interstellar  2014       11187   
2843                                          Fight Club  1999        9678   
4863   The Lord of the Rings: The Fellowship of the Ring  2001        8892   
292                                         Pulp Fiction  1994        8670   
314                             The Shawshank Redemption  1994        8358   
7000       The Lord of the Rings: The Return of the King  2003        8226   
351                                         Forrest Gump  1994        8147   
5814               The Lord of the Rings: The Two Towers  2002        7641   
256                                            Star Wars  1977        6778   
1225                                  Back to the Future  1985        6239   
834                                        The Godfather  1972        6024   
1154                             The Empire Strikes Back  1980        5998   
46                                                 Se7en  1995        5915   

       vote_average  popularity  \
15480             8   29.108149   
12481             8  123.167259   
22879             8   32.213481   
2843              8   63.869599   
4863              8   32.070725   
292               8  140.950236   
314               8   51.645403   
7000              8   29.324358   
351               8   48.307194   
5814              8   29.423537   
256               8   42.149697   
1225              8   25.778509   
834               8   41.109264   
1154              8   19.470959   
46                8    18.45743   

                                                  genres        wr  
15480  [Action, Thriller, Science Fiction, Mystery, A...  7.917588  
12481                   [Drama, Action, Crime, Thriller]  7.905871  
22879                [Adventure, Drama, Science Fiction]  7.897107  
2843                                             [Drama]  7.881753  
4863                        [Adventure, Fantasy, Action]  7.871787  
292                                    [Thriller, Crime]  7.868660  
314                                       [Drama, Crime]  7.864000  
7000                        [Adventure, Fantasy, Action]  7.861927  
351                             [Comedy, Drama, Romance]  7.860656  
5814                        [Adventure, Fantasy, Action]  7.851924  
256                 [Adventure, Action, Science Fiction]  7.834205  
1225        [Adventure, Comedy, Science Fiction, Family]  7.820813  
834                                       [Drama, Crime]  7.814847  
1154                [Adventure, Action, Science Fiction]  7.814099  
46                            [Crime, Mystery, Thriller]  7.811669

* We see that three Christopher Nolan Films, __Inception__, __The Dark Knight__ and __Interstellar__ occur at the very top of our chart. 
* The chart also indicates a strong bias of TMDB Users towards particular genres and directors.

* Let us now construct our __function that builds charts for particular genres.__

* For this, we __relax__ our __default conditions to the 85th percentile instead of 95.__

In [47]:
'''
>>> s
     a   b
one  1.  2.
two  3.  4.

>>> s.stack()
one a    1
    b    2
two a    3
    b    4
'''
s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
gen_md = md.drop('genres', axis=1).join(s)
gen_md.head(3).transpose()

C:\Users\91720\AppData\Local\Temp\ipykernel_11760\2441948897.py:13: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  s = md.apply(lambda x: pd.Series(x['genres']),axis=1).stack().reset_index(level=1, drop=True)


0  \
adult                                                              False   
belongs_to_collection  {'id': 10194, 'name': 'Toy Story Collection', ...   
budget                                                          30000000   
homepage                            http://toystory.disney.com/toy-story   
id                                                                   862   
imdb_id                                                        tt0114709   
original_language                                                     en   
original_title                                                 Toy Story   
overview               Led by Woody, Andy's toys live happily in his ...   
popularity                                                     21.946943   
poster_path                             /rhIRbceoE9lR4veEXuwCC2wARtG.jpg   
production_companies      [{'name': 'Pixar Animation Studios', 'id': 3}]   
production_countries   [{'iso_3166_1': 'US', 'name': 'United States o...   
release_date                                                  1995-10-30   
revenue                                                      373554033.0   
runtime                                                             81.0   
spoken_languages                [{'iso_639_1': 'en', 'name': 'English'}]   
status                                                          Released   
tagline                                                              NaN   
title                                                          Toy Story   
video                                                              False   
vote_average                                                         7.7   
vote_count                                                        5415.0   
year                                                                1995   
genre                                                          Animation   

                                                                       0  \
adult                                                              False   
belongs_to_collection  {'id': 10194, 'name': 'Toy Story Collection', ...   
budget                                                          30000000   
homepage                            http://toystory.disney.com/toy-story   
id                                                                   862   
imdb_id                                                        tt0114709   
original_language                                                     en   
original_title                                                 Toy Story   
overview               Led by Woody, Andy's toys live happily in his ...   
popularity                                                     21.946943   
poster_path                             /rhIRbceoE9lR4veEXuwCC2wARtG.jpg   
production_companies      [{'name': 'Pixar Animation Studios', 'id': 3}]   
production_countries   [{'iso_3166_1': 'US', 'name': 'United States o...   
release_date                                                  1995-10-30   
revenue                                                      373554033.0   
runtime                                                             81.0   
spoken_languages                [{'iso_639_1': 'en', 'name': 'English'}]   
status                                                          Released   
tagline                                                              NaN   
title                                                          Toy Story   
video                                                              False   
vote_average                                                         7.7   
vote_count                                                        5415.0   
year                                                                1995   
genre                                                             Comedy   

                                                                       0  
adult                                     

In [52]:
def build_chart(genre, percentile=0.85):
    df = gen_md[gen_md['genre'] == genre]
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) & (df['vote_count'].notnull()) & 
                   (df['vote_average'].notnull())][['title', 'year', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    
    qualified['wr'] = qualified.apply(lambda x: 
                        (x['vote_count']/(x['vote_count']+m) * x['vote_average']) + (m/(m+x['vote_count']) * C),
                        axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(250)
    
    return qualified



**Top 15 Romantic Movies**

In [53]:
build_chart('Romance').head(15)

title  year  vote_count  vote_average popularity  \
10309  Dilwale Dulhania Le Jayenge  1995         661             9  34.457024   
351                   Forrest Gump  1994        8147             8  48.307194   
876                        Vertigo  1958        1162             8   18.20822   
40251                   Your Name.  2016        1030             8  34.461252   
883               Some Like It Hot  1959         835             8  11.845107   
1132               Cinema Paradiso  1988         834             8  14.177005   
19901                     Paperman  2012         734             8   7.198633   
37863                  Sing Street  2016         669             8  10.672862   
882                  The Apartment  1960         498             8  11.994281   
38718               The Handmaiden  2016         453             8  16.727405   
3189                   City Lights  1931         444             8  10.891524   
24886             The Way He Looks  2014         262             8   5.711274   
45437               In a Heartbeat  2017         146             8   20.82178   
1639                       Titanic  1997        7770             7   26.88907   
19731      Silver Linings Playbook  2012        4840             7  14.488111   

             wr  
10309  8.565285  
351    7.971357  
876    7.811667  
40251  7.789489  
883    7.745154  
1132   7.744878  
19901  7.713951  
37863  7.689483  
882    7.599317  
38718  7.566166  
3189   7.558867  
24886  7.331363  
45437  7.003959  
1639   6.981546  
19731  6.970581

###  Content based recommendation system

In [ ]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [ ]:
## Pre-processing step

def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan


In [55]:
md['id'] = md['id'].apply(convert_int)
md[md['id'].isnull()]

adult  \
19730                                 - Written by Ørnås   
29503   Rune Balot goes to a casino connected to the ...   
35587   Avalanche Sharks tells the story of a bikini ...   

      belongs_to_collection                            budget  \
19730              0.065736  /ff9qCepilowshEtG2GYWwzt2bs4.jpg   
29503              1.931659  /zV8bHuSL6WXoD6FWogP9j4x80bL.jpg   
35587              2.185485  /zaSf5OG7V8X8gqFvly88zDdRm46.jpg   

                                                  genres  \
19730  [Carousel Productions, Vision View Entertainme...   
29503  [Aniplex, GoHands, BROSTA TV, Mardock Scramble...   
35587  [Odyssey Media, Pulser Productions, Rogue Stat...   

                                                homepage  id imdb_id  \
19730  [{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso... NaN       0   
29503  [{'iso_3166_1': 'US', 'name': 'United States o... NaN       0   
35587           [{'iso_3166_1': 'CA', 'name': 'Canada'}] NaN       0   

      original_language                            original_title  overview  \
19730             104.0  [{'iso_639_1': 'en', 'name': 'English'}]  Released   
29503              68.0      [{'iso_639_1': 'ja', 'name': '日本語'}]  Released   
35587              82.0  [{'iso_639_1': 'en', 'name': 'English'}]  Released   

       ... revenue runtime spoken_languages status tagline  title  video  \
19730  ...     NaN     NaN              NaN    NaN     NaN    NaN    NaN   
29503  ...     NaN     NaN              NaN    NaN     NaN    NaN    NaN   
35587  ...     NaN     NaN              NaN    NaN     NaN    NaN    NaN   

      vote_average vote_count year  
19730          NaN        NaN  NaT  
29503          NaN        NaN  NaT  
35587          NaN        NaN  NaT  

[3 rows x 25 columns]

In [56]:
md = md.drop([19730, 29503, 35587])

In [57]:
md['id'] = md['id'].astype('int')

In [58]:
smd = md[md['id'].isin(links_small)]
smd.shape


(9099, 25)

We have __9099 movies__ available in our small movies metadata dataset which is 5 times smaller than our original dataset of 45000 movies.

In [59]:
smd['tagline'] = smd['tagline'].fillna('')
smd['description'] = smd['overview'] + smd['tagline']
smd['description'] = smd['description'].fillna('')

C:\Users\91720\AppData\Local\Temp\ipykernel_11760\3912728496.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['tagline'] = smd['tagline'].fillna('')
C:\Users\91720\AppData\Local\Temp\ipykernel_11760\3912728496.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  smd['description'] = smd['overview'] + smd['tagline']
C:\Users\91720\AppData\Local\Temp\ipykernel_11760\3912728496.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [ ]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(smd['description'])

In [ ]:
tfidf_matrix.shape

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim[0]

In [48]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])
#indices.head(2)

In [49]:
def get_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

* We're all set...!
* Let us now try and get the top recommendations for a few movies and see how good the recommendations are.

In [50]:
get_recommendations('The Godfather').head(10)

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
29               Shanghai Triad
5667                       Fury
2412             American Movie
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
Name: title, dtype: object

In [51]:
get_recommendations('The Dark Knight').head(10)

7931                      The Dark Knight Rises
132                              Batman Forever
1113                             Batman Returns
8227    Batman: The Dark Knight Returns, Part 2
7565                 Batman: Under the Red Hood
524                                      Batman
7901                           Batman: Year One
2579               Batman: Mask of the Phantasm
2696                                        JFK
8165    Batman: The Dark Knight Returns, Part 1
Name: title, dtype: object

### Content based RS : Using movie description, taglines, keywords, cast, director and genres

* To build our standard metadata based content recommender, we will need to __merge our current dataset with the 
  crew and the keyword datasets__. 
* Let us prepare this data as our first step.

In [53]:
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
md['id'] = md['id'].astype('int')

In [54]:
md.shape

(45463, 25)

In [55]:
md = md.merge(credits, on='id')
md = md.merge(keywords, on='id')

In [56]:
smd = md[md['id'].isin(links_small)]
smd.shape

# smd = md[md['id'].isin(links_small['tmdbId'])]
# smd.shape

(9219, 28)

In [57]:
smd['cast'] = smd['cast'].apply(literal_eval)
smd['crew'] = smd['crew'].apply(literal_eval)
smd['keywords'] = smd['keywords'].apply(literal_eval)
smd['cast_size'] = smd['cast'].apply(lambda x: len(x))
smd['crew_size'] = smd['crew'].apply(lambda x: len(x))

In [58]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [59]:
smd['director'] = smd['crew'].apply(get_director)
smd['cast'] = smd['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
smd['cast'] = smd['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)
smd['keywords'] = smd['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [61]:
smd['cast'] = smd['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])
smd['director'] = smd['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
smd['director'] = smd['director'].apply(lambda x: [x,x, x])

In [64]:
s = smd.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [65]:
s = s[s > 1]

In [66]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [67]:
def filter_keywords(x):
    words = []
    for i in x:
        if i in s:
            words.append(i)
    return words

In [69]:
smd['keywords'] = smd['keywords'].apply(filter_keywords)
smd['keywords'] = smd['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
smd['keywords'] = smd['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [70]:
smd['soup'] = smd['keywords'] + smd['cast'] + smd['director'] + smd['genres']
smd['soup'] = smd['soup'].apply(lambda x: ' '.join(x))

In [71]:
count = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
count_matrix = count.fit_transform(smd['soup'])

In [72]:
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [73]:
smd = smd.reset_index()
titles = smd['title']
indices = pd.Series(smd.index, index=smd['title'])

In [74]:
get_recommendations('The Dark Knight').head(10)

6218              Batman Begins
6623               The Prestige
8031      The Dark Knight Rises
2085                  Following
7648                  Inception
4145                   Insomnia
3381                    Memento
8613               Interstellar
8927    Kidnapping Mr. Heineken
5943                   Thursday
Name: title, dtype: object

In [75]:
get_recommendations('Inception').head(10)

6623             The Prestige
6218            Batman Begins
4145                 Insomnia
2085                Following
8613             Interstellar
6981          The Dark Knight
3381                  Memento
8031    The Dark Knight Rises
8500                  Don Jon
8207                   Looper
Name: title, dtype: object

In [76]:
get_recommendations('Mean Girls').head(10)

1329                        The House of Yes
3319                         Head Over Heels
4763                           Freaky Friday
7905                   Mr. Popper's Penguins
7332              Ghosts of Girlfriends Past
6277                        Just Like Heaven
6959               The Spiderwick Chronicles
5152    Confessions of a Teenage Drama Queen
1637                         The Parent Trap
3178                          The Ladies Man
Name: title, dtype: object

In [77]:
get_recommendations('Pulp Fiction').head(10)

5200                               Kill Bill: Vol. 2
1381                                    Jackie Brown
8905                               The Hateful Eight
898                                   Reservoir Dogs
4903                               Kill Bill: Vol. 1
8310                                Django Unchained
7280                            Inglourious Basterds
6788                                     Death Proof
65                               From Dusk Till Dawn
6392    The Cutting Edge: The Magic of Movie Editing
Name: title, dtype: object

#### Add Popularity and Ratings 

In [79]:
def improved_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year']]
    vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = movies[movies['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(0.60)
    qualified = movies[(movies['vote_count'] >= m) & (movies['vote_count'].notnull()) & 
                       (movies['vote_average'].notnull())]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['wr'] = qualified.apply(weighted_rating, axis=1)
    qualified = qualified.sort_values('wr', ascending=False).head(10)
    return qualified

In [80]:
improved_recommendations('The Dark Knight')

title  vote_count  vote_average  year        wr
7648              Inception       14075             8  2010  7.917588
8613           Interstellar       11187             8  2014  7.897107
6623           The Prestige        4510             8  2006  7.758148
3381                Memento        4168             8  2000  7.740175
8031  The Dark Knight Rises        9263             7  2012  6.921448
6218          Batman Begins        7511             7  2005  6.904127
1532  The French Connection         435             7  1971  6.123458
4145               Insomnia        1181             6  2002  5.797081
7133             RockNRolla         851             6  2008  5.744969
8026     Bullet to the Head         490             5  2013  5.115027

In [81]:
improved_recommendations('Pulp Fiction')

title  vote_count  vote_average  year        wr
898         Reservoir Dogs        3821             8  1992  7.718986
8310      Django Unchained       10297             7  2012  6.929017
7280  Inglourious Basterds        6598             7  2009  6.891679
4903     Kill Bill: Vol. 1        5091             7  2003  6.862133
8905     The Hateful Eight        4405             7  2015  6.842588
5200     Kill Bill: Vol. 2        4061             7  2004  6.830542
1381          Jackie Brown        1580             7  1997  6.621790
65     From Dusk Till Dawn        1644             6  1996  5.842293
6788           Death Proof        1359             6  2007  5.817225
4764              S.W.A.T.         780             5  2003  5.087550

In [85]:
reader = Reader()

In [86]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
data.split(n_folds=5)

In [87]:
svd = SVD()
evaluate(svd, data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SVD.

------------
Fold 1
RMSE: 0.9061
MAE:  0.6978
------------
Fold 2
RMSE: 0.8942
MAE:  0.6894
------------
Fold 3
RMSE: 0.8924
MAE:  0.6856
------------
Fold 4
RMSE: 0.8983
MAE:  0.6902
------------
Fold 5
RMSE: 0.8950
MAE:  0.6911
------------
------------
Mean RMSE: 0.8972
Mean MAE : 0.6908
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [0.6977986010578162,
                             0.6893989097172819,
                             0.6855953736686582,
                             0.6902436024123828,
                             0.6910891113934926],
                            'rmse': [0.9061093999472343,
                             0.894157877002033,
                             0.8924156875301242,
                             0.8983103096964073,
                             0.8949992619618011]})

In [88]:
trainset = data.build_full_trainset()
svd.train(trainset)

In [89]:
ratings[ratings['userId'] == 1]

userId  movieId  rating   timestamp
0        1       31     2.5  1260759144
1        1     1029     3.0  1260759179
2        1     1061     3.0  1260759182
3        1     1129     2.0  1260759185
4        1     1172     4.0  1260759205
5        1     1263     2.0  1260759151
6        1     1287     2.0  1260759187
7        1     1293     2.0  1260759148
8        1     1339     3.5  1260759125
9        1     1343     2.0  1260759131
10       1     1371     2.5  1260759135
11       1     1405     1.0  1260759203
12       1     1953     4.0  1260759191
13       1     2105     4.0  1260759139
14       1     2150     3.0  1260759194
15       1     2193     2.0  1260759198
16       1     2294     2.0  1260759108
17       1     2455     2.5  1260759113
18       1     2968     1.0  1260759200
19       1     3671     3.0  1260759117

In [112]:
svd.predict(1, 302)

Prediction(uid=1, iid=302, r_ui=None, est=2.7480119426314853, details={'was_impossible': False})

### Hybrid recommendation system

* **Input:** User ID and the Title of a Movie
* **Output:** Similar movies sorted on the basis of expected ratings by that particular user.

In [94]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [95]:
id_map = pd.read_csv('./input_data/movie_dataset/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(smd[['title', 'id']], on='id').set_index('title')
#id_map = id_map.set_index('tmdbId')

In [96]:
indices_map = id_map.set_index('id')

In [103]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']
    sim_scores = list(enumerate(cosine_sim[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    movies = smd.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'release_date', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [104]:
hybrid(1, 'Avatar')

title  vote_count  vote_average release_date  \
974                           Aliens      3282.0           7.7   1986-07-18   
1011                  The Terminator      4208.0           7.4   1984-10-26   
8658      X-Men: Days of Future Past      6155.0           7.5   2014-05-15   
522       Terminator 2: Judgment Day      4274.0           7.7   1991-07-01   
8401         Star Trek Into Darkness      4479.0           7.4   2013-05-05   
344                        True Lies      1138.0           6.8   1994-07-14   
2014                Fantastic Planet       140.0           7.6   1973-05-01   
922                        The Abyss       822.0           7.1   1989-08-09   
4347  Piranha Part Two: The Spawning        41.0           3.9   1981-01-01   
8231                    Side by Side       111.0           7.1   2012-08-19   

          id       est  
974      679  3.343526  
1011     218  3.165331  
8658  127585  3.095831  
522      280  3.005560  
8401   54138  2.845466  
344    36955  2.840542  
2014   16306  2.820183  
922     2756  2.722440  
4347   31646  2.684467  
8231  110354  2.647725

In [105]:
hybrid(5000, 'Avatar')


title  vote_count  vote_average release_date  \
1011              The Terminator      4208.0           7.4   1984-10-26   
522   Terminator 2: Judgment Day      4274.0           7.7   1991-07-01   
974                       Aliens      3282.0           7.7   1986-07-18   
8401     Star Trek Into Darkness      4479.0           7.4   2013-05-05   
8658  X-Men: Days of Future Past      6155.0           7.5   2014-05-15   
2014            Fantastic Planet       140.0           7.6   1973-05-01   
8231                Side by Side       111.0           7.1   2012-08-19   
922                    The Abyss       822.0           7.1   1989-08-09   
344                    True Lies      1138.0           6.8   1994-07-14   
5310        Frank Herbert's Dune       114.0           6.7   2000-12-03   

          id       est  
1011     218  3.956651  
522      280  3.951281  
974      679  3.902304  
8401   54138  3.789442  
8658  127585  3.785119  
2014   16306  3.718719  
8231  110354  3.634942  
922     2756  3.614626  
344    36955  3.549296  
5310     876  3.483959

In [111]:
hybrid(3423, "The Terminator")

title  vote_count  vote_average release_date  \
522   Terminator 2: Judgment Day      4274.0           7.7   1991-07-01   
974                       Aliens      3282.0           7.7   1986-07-18   
2972                     Mad Max      1235.0           6.6   1979-04-12   
7502             The Book of Eli      2207.0           6.6   2010-01-14   
4173             Minority Report      2663.0           7.1   2002-06-20   
6394                District B13       572.0           6.5   2004-11-09   
8231                Side by Side       111.0           7.1   2012-08-19   
7488                      Avatar     12114.0           7.2   2009-12-10   
922                    The Abyss       822.0           7.1   1989-08-09   
6622             Children of Men      2120.0           7.4   2006-09-22   

          id       est  
522      280  3.951281  
974      679  3.902304  
2972    9659  3.838765  
7502   20504  3.809446  
4173     180  3.758776  
6394   10045  3.703780  
8231  110354  3.634942  
7488   19995  3.615983  
922     2756  3.614626  
6622    9693  3.614090